In [1]:
import pandas as pd
import json
import os

# Ruta de los archivos
base_path = os.path.abspath(os.getcwd())
path = os.path.join(base_path, 'data')

# Carga de archivos
df_users = pd.read_csv(os.path.join(path, 'users_data.csv'))
df_cards = pd.read_csv(os.path.join(path, 'cards_data.csv'))
df_transactions = pd.read_csv(os.path.join(path, 'transactions_data.csv'))

with open(os.path.join(path, 'mcc_codes.json'), 'r') as file:
    mcc_json = json.load(file)
df_mcc = pd.DataFrame(mcc_json.items(), columns=['mcc', 'description'])

with open(os.path.join(path, 'train_fraud_labels.json'), 'r') as file:
    fraud_json = json.load(file)
df_fraud_labels = pd.DataFrame(fraud_json['target'].items(), columns=['transaction_id', 'is_fraud'])




In [2]:
# Filtrar edades entre 18 y 75
df_users = df_users[(df_users['current_age'] >= 18) & (df_users['current_age'] <= 75)]

# Eliminar '$' y convertir a float
cols_monetarias = ['per_capita_income', 'yearly_income', 'total_debt']
for col in cols_monetarias:
    df_users[col] = df_users[col].replace('[\$,]', '', regex=True).astype(float)

# Filtrar solo registros con deuda positiva
df_users = df_users[df_users['total_debt'] > 0]

# Eliminar duplicados en id
df_users = df_users.drop_duplicates(subset='id')

# Renombrar PK
df_users = df_users.rename(columns={'id': 'cliente_id'})

# Seleccionar solo columnas necesarias
df_users = df_users[[
    'cliente_id', 'current_age', 'gender',
    'per_capita_income', 'yearly_income', 'total_debt',
    'credit_score', 'num_credit_cards'
]]

<>:7: SyntaxWarning: invalid escape sequence '\$'
<>:7: SyntaxWarning: invalid escape sequence '\$'
C:\Users\samue\AppData\Local\Temp\ipykernel_6984\3828758017.py:7: SyntaxWarning: invalid escape sequence '\$'
  df_users[col] = df_users[col].replace('[\$,]', '', regex=True).astype(float)


In [3]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1755 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   cliente_id         1755 non-null   int64  
 1   current_age        1755 non-null   int64  
 2   gender             1755 non-null   object 
 3   per_capita_income  1755 non-null   float64
 4   yearly_income      1755 non-null   float64
 5   total_debt         1755 non-null   float64
 6   credit_score       1755 non-null   int64  
 7   num_credit_cards   1755 non-null   int64  
dtypes: float64(3), int64(4), object(1)
memory usage: 123.4+ KB


In [4]:
df_users.head()

,cliente_id,current_age,gender,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,Female,29278.0,59696.0,127613.0,787,5
1,1746,53,Female,37891.0,77254.0,191349.0,701,5
3,708,63,Female,163145.0,249925.0,202328.0,722,4
4,1164,43,Male,53797.0,109687.0,183855.0,675,1
6,1075,36,Female,25258.0,51500.0,102286.0,672,3


In [5]:
# Eliminar duplicados
df_cards = df_cards.drop_duplicates(subset='id')

# Eliminar registros sin card_brand o card_type
df_cards = df_cards.dropna(subset=['card_brand', 'card_type'])

# Eliminar '$' de credit_limit y convertir a float
df_cards['credit_limit'] = df_cards['credit_limit'].replace('[\$,]', '', regex=True).astype(float)

# Filtrar credit_limit > 100
df_cards = df_cards[df_cards['credit_limit'] > 100]

# Renombrar PK y FK
df_cards = df_cards.rename(columns={'id': 'tarjeta_id', 'client_id': 'cliente_id'})

# Seleccionar columnas finales
df_cards = df_cards[[
    'tarjeta_id', 'cliente_id', 'card_brand', 'card_type',
    'num_cards_issued', 'credit_limit', 'card_on_dark_web'
]]


<>:8: SyntaxWarning: invalid escape sequence '\$'
<>:8: SyntaxWarning: invalid escape sequence '\$'
C:\Users\samue\AppData\Local\Temp\ipykernel_6984\2584924708.py:8: SyntaxWarning: invalid escape sequence '\$'
  df_cards['credit_limit'] = df_cards['credit_limit'].replace('[\$,]', '', regex=True).astype(float)


In [8]:
# Eliminar duplicados por id
df_transactions = df_transactions.drop_duplicates(subset='id')

# Filtrar transacciones con amount positivo
df_transactions = df_transactions[df_transactions['amount'].str.replace('$', '', regex=False).astype(float) > 0]

# Eliminar símbolo '$' en amount y convertir a float
df_transactions['amount'] = df_transactions['amount'].str.replace('$', '', regex=False).astype(float)

# Rellenar valores nulos o vacíos en errors con 'N/A'
df_transactions['errors'] = df_transactions['errors'].fillna('N/A')
df_transactions['errors'] = df_transactions['errors'].replace('', 'N/A')

# Convertir id a string para merge
df_transactions['id'] = df_transactions['id'].astype(str)
df_fraud_labels['transaction_id'] = df_fraud_labels['transaction_id'].astype(str)

# Cruzar con df_fraud_labels y traer is_fraud
df_transactions = df_transactions.merge(
    df_fraud_labels,
    left_on='id',
    right_on='transaction_id',
    how='inner'
)

# Eliminar columna innecesaria
df_transactions = df_transactions.drop(columns=['transaction_id'])

# Seleccionar solo las columnas necesarias
df_transactions = df_transactions[[
    'id', 'date', 'client_id', 'card_id', 'amount',
    'merchant_id', 'merchant_city', 'merchant_state',
    'mcc', 'errors', 'is_fraud'
]]


In [9]:
df_transactions = df_transactions.dropna(subset=['merchant_id', 'merchant_city', 'merchant_state', 'client_id', 'card_id'])


In [10]:
# Convertir mcc de ambos DF a string
df_transactions['mcc'] = df_transactions['mcc'].astype(str)
df_mcc['mcc'] = df_mcc['mcc'].astype(str)

# Cruzar solo con mcc que existen en df_mcc
df_transactions = df_transactions.merge(df_mcc, how='inner', on='mcc')


In [11]:
# Cruzar tarjetas y clientes, solo los que existen en ambos
df_cards_users = df_cards.merge(df_users, how='inner', on='cliente_id', suffixes=('_card', '_user'))


In [12]:
df_cards_users.head()

,tarjeta_id,cliente_id,card_brand,card_type,num_cards_issued,credit_limit,card_on_dark_web,current_age,gender,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,4524,825,Visa,Debit,2,24295.0,No,53,Female,29278.0,59696.0,127613.0,787,5
1,2731,825,Visa,Debit,2,21968.0,No,53,Female,29278.0,59696.0,127613.0,787,5
2,3701,825,Visa,Debit,2,46414.0,No,53,Female,29278.0,59696.0,127613.0,787,5
3,42,825,Visa,Credit,1,12400.0,No,53,Female,29278.0,59696.0,127613.0,787,5
4,4537,1746,Visa,Credit,1,27500.0,No,53,Female,37891.0,77254.0,191349.0,701,5


In [13]:
df_trx_cards_users = df_transactions.merge(
    df_cards_users,
    how='inner',
    left_on=['client_id', 'card_id'],
    right_on=['cliente_id', 'tarjeta_id']
)

In [14]:
# Eliminar columnas duplicadas luego del cruce
df_trx_cards_users = df_trx_cards_users.drop(columns=['cliente_id', 'tarjeta_id'])

In [15]:
df_trx_cards_users = df_trx_cards_users.astype({
    'client_id': 'int32',
    'card_id': 'int32',
    'amount': 'float32',
    'merchant_id': 'int32',
    'num_cards_issued': 'int8',
    'credit_limit': 'float32',
    'current_age': 'int8',
    'per_capita_income': 'float32',
    'yearly_income': 'float32',
    'total_debt': 'float32',
    'credit_score': 'int16',
    'num_credit_cards': 'int8'
})

In [16]:
df_trx_cards_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5740339 entries, 0 to 5740338
Data columns (total 24 columns):
 #   Column             Dtype  
---  ------             -----  
 0   id                 object 
 1   date               object 
 2   client_id          int32  
 3   card_id            int32  
 4   amount             float32
 5   merchant_id        int32  
 6   merchant_city      object 
 7   merchant_state     object 
 8   mcc                object 
 9   errors             object 
 10  is_fraud           object 
 11  description        object 
 12  card_brand         object 
 13  card_type          object 
 14  num_cards_issued   int8   
 15  credit_limit       float32
 16  card_on_dark_web   object 
 17  current_age        int8   
 18  gender             object 
 19  per_capita_income  float32
 20  yearly_income      float32
 21  total_debt         float32
 22  credit_score       int16  
 23  num_credit_cards   int8   
dtypes: float32(5), int16(1), int32(3), int8(3), object

In [17]:
df_trx_cards_users.head()

,id,date,client_id,card_id,amount,merchant_id,merchant_city,merchant_state,mcc,errors,...,num_cards_issued,credit_limit,card_on_dark_web,current_age,gender,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,7475328,2010-01-01 00:02:00,561,4575,14.57,67570,Bettendorf,IA,5311,N/A,...,1,9100.0,No,48,Male,18076.0,36853.0,112139.0,834,5
1,7475329,2010-01-01 00:02:00,1129,102,80.00,27092,Vista,CA,4829,N/A,...,1,14802.0,No,49,Male,16894.0,34449.0,36540.0,686,3
2,7475332,2010-01-01 00:06:00,848,3915,46.41,13051,Harwood,MD,5813,N/A,...,1,19113.0,No,51,Male,33529.0,68362.0,96182.0,711,2
3,7475338,2010-01-01 00:23:00,554,3912,3.51,67570,Pearland,TX,5311,N/A,...,1,25658.0,No,59,Male,26170.0,53357.0,114266.0,690,5
4,7475339,2010-01-01 00:23:00,605,5061,2.58,75781,Brooklyn,NY,5411,N/A,...,2,1484.0,No,42,Male,23316.0,47542.0,2667.0,725,3


In [19]:
import pandas as pd
import sqlite3


df_trx_cards_users['date'] = pd.to_datetime(df_trx_cards_users['date']).dt.strftime('%Y%m%d').astype(int)
# Crear conexión a base de datos SQLite
conn = sqlite3.connect('datawarehouse_final.db')

print('Inicia proceso de guardado')

# Guardar dataframe en SQLite
df_trx_cards_users.to_sql('transactions_dw', conn, if_exists='replace', index=False)

print('Guardado exitoso, iniciando optimización...')

# Optimización del archivo SQLite
conn.execute("VACUUM;")
conn.execute("PRAGMA optimize;")

conn.close()

print('Exportación completada y base de datos optimizada. Archivo: datawarehouse_final.db')


Inicia proceso de guardado
Guardado exitoso, iniciando optimización...
Exportación completada y base de datos optimizada. Archivo: datawarehouse_final.db
